- index能够简化查询
- index能够优化查询性能
    - 如果index是唯一的，那么index会使用hash优化算法，时间复杂的为O(1)
    - 如果index不是唯一的，但是是有序的，那么index会使用二分查找来优化，时间复杂度为O(Log(N))
    - 如果index无须切不是唯一，那么需要全表扫描，时间复杂度为O(N)
- index能够数据对齐

In [53]:
import pandas as pd
import numpy as np
df=pd.DataFrame(
    np.arange(300000).reshape(100000, 3),
    columns=["userId","moveId","star"]
)
# 增加一行, 使得不是唯一
df.loc[100000]=[0,1,3]


#### 1. 简化查询演示(查找userId为1000的数据)

In [54]:
df.head(1)

,userId,moveId,star
0,0,1,2


In [55]:
# df["userId"].value_counts()
# a. 间接查询
df.loc[df.userId==0,:]

,userId,moveId,star
0,0,1,2
100000,0,1,3


In [56]:
# b. 直接查询
# b.1 设置索引, implace: 原地修改; drop: 是否丢弃原df中的userId列
df.set_index("userId",inplace=True, drop=False)

In [57]:
df.head(2)

,userId,moveId,star
userId,,,
0,0,1,2
3,3,4,5


In [58]:
df.loc[0,:]

,userId,moveId,star
userId,,,
0,0,1,2
0,0,1,3


#### 2. 优化性能

In [59]:
from sklearn.utils import shuffle
df_shuffle=shuffle(df)

In [60]:
df_shuffle.head(1)

,userId,moveId,star
userId,,,
10839,10839,10840,10841


In [61]:
# index 是否单调递增
df_shuffle.index.is_monotonic_increasing

False

In [62]:
# index 是否唯一
df_shuffle.index.is_unique

False

In [72]:
# 1. 不唯一、无序的df 耗时
%timeit df_shuffle.loc[224061]

73.7 µs ± 451 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [65]:
df_sorted = df_shuffle.sort_index()

In [66]:
df_sorted.index.is_monotonic_increasing

True

In [67]:
df_sorted.index.is_unique

False

In [73]:
# 2. 不唯一、有序的df 耗时
%timeit df_shuffle.loc[224061]

78.2 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [75]:
# 2. 唯一、无序的df 耗时
%timeit df.loc[224061]

246 µs ± 129 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### 3. 数据对齐

In [77]:
s1=pd.Series([1,2,3],list("abc"))

In [78]:
s1

a    1
b    2
c    3
dtype: int64

In [79]:
s2=pd.Series([2,3,4],list("bcd"))
s2

b    2
c    3
d    4
dtype: int64

In [80]:
s1+s2

a    NaN
b    4.0
c    6.0
d    NaN
dtype: float64